In [1]:
import pandas as pd
import numpy as np
import sqlite3

connection = sqlite3.connect('books.db')
cursor = connection.cursor()

def select(query):
    try:
        return pd.read_sql_query(query, connection)
    except:
        cursor.executescript(query)
        connection.commit()


In [2]:
query = r'''
    SELECT *
    FROM city
    LIMIT 5
'''
select(query).columns

Index(['address', 'postal_code', 'country', 'federal_district', 'region_type',
       'region', 'area_type', 'area', 'city_type', 'city', 'settlement_type',
       'settlement', 'kladr_id', 'fias_id', 'fias_level', 'capital_marker',
       'okato', 'oktmo', 'tax_office', 'timezone', 'geo_lat', 'geo_lon',
       'population', 'foundation_year'],
      dtype='object')

In [8]:
query = r'''
    SELECT 
        timezone,
        COUNT(*) AS city_count
    FROM city
    WHERE federal_district LIKE '%Сибирск%' OR federal_district LIKE '%Приволжск%'
    GROUP BY timezone
'''
select(query)

,timezone,city_count
0,UTC+3,101
1,UTC+4,41
2,UTC+5,58
3,UTC+6,6
4,UTC+7,86
5,UTC+8,22


<h1>Ближайшие города</h1>
<div class = 'alert alert-block alert-success'>
    Напишите запрос, который найдет три ближайших к Самаре города, не считая саму Самару.
</div>

In [12]:
query = r'''
    WITH samara_coord AS (
    SELECT 
        geo_lat AS samara_lat,
        geo_lon AS samara_lon
    FROM city
    WHERE city = 'Самара'
    )
    SELECT 
        DISTINCT city
    FROM city, samara_coord
    WHERE city <> 'Самара'
    ORDER BY SQRT(POW(samara_lat - geo_lat, 2) + POW(samara_lon - geo_lon, 2))
    LIMIT 3
'''
select(query)

,city
0,Новокуйбышевск
1,Чапаевск
2,Кинель


In [6]:
pd.read_csv('samara.csv')

,kladr_id,city
0,6300000200000,Жигулевск
1,6300001000000,Кинель
2,6301700100000,Нефтегорск
3,6300000300000,Новокуйбышевск
4,6300000400000,Октябрьск
5,6300000500000,Отрадный
6,6300000900000,Похвистнево
7,6300000100000,Самара
8,6300000800000,Сызрань
9,6300000700000,Тольятти


<H1>
Выгружаем результаты запроса    
</H1> 

Напишите запрос, который посчитает количество городов в каждом часовом поясе. Отсортируйте по количеству городов по убыванию. Получится примерно так:

In [9]:
query = r'''
    SELECT 
        timezone,
        COUNT(*) AS city_count
    FROM city
    GROUP BY timezone
    ORDER BY city_count DESC
'''
select(query)

,timezone,city_count
0,UTC+3,660
1,UTC+5,173
2,UTC+7,86
3,UTC+4,66
4,UTC+9,31
5,UTC+8,28
6,UTC+2,22
7,UTC+10,22
8,UTC+11,17
9,UTC+6,6


In [14]:
pd.read_csv('books.csv').to_sql('books', connection, index=False, if_exists = 'replace')

1500

In [15]:
query = r'''
    SELECT 
        COUNT(*)
    FROM books
'''
select(query)

,COUNT(*)
0,1500


In [20]:
query = r'''
    PRAGMA TABLE_INFO(books)
'''
select(query)

,cid,name,type,notnull,dflt_value,pk
0,0,book_id,INTEGER,0,None,0
1,1,title,TEXT,0,None,0
2,2,authors,TEXT,0,None,0
3,3,average_rating,REAL,0,None,0
4,4,isbn,TEXT,0,None,0
5,5,isbn13,INTEGER,0,None,0
6,6,language_code,TEXT,0,None,0
7,7,num_pages,INTEGER,0,None,0
8,8,ratings_count,INTEGER,0,None,0
9,9,text_reviews_count,INTEGER,0,None,0


In [26]:
query = r'''
    WITH get_max_ratings_count AS (
        SELECT MAX(ratings_count) AS max_ratings_count
        FROM books
    )
    SELECT book_id
    FROM books, get_max_ratings_count
    WHERE ratings_count = max_ratings_count
'''
select(query)

,book_id
0,5107


In [27]:
query = r'''
    WITH get_max_av_rating AS (
    SELECT
        MAX(average_rating) AS max_av_rating
    FROM books
    WHERE ratings_count > 100
    )
    SELECT 
        book_id
    FROM books, get_max_av_rating
    WHERE ratings_count > 100 AND average_rating = max_av_rating
'''
select(query)

,book_id
0,8


In [28]:
query = r'''
    SELECT 
        ROUND(AVG(average_rating), 2) AS av_rating
    FROM books
    WHERE ratings_count > 100
'''
select(query)

,av_rating
0,3.95


In [34]:
query = r'''
    UPDATE books
    SET language_code = 'eng'
    WHERE language_code IN ('en-CA', 'en-GB', 'en-US', 'enm')
'''
select(query)
connection.commit()

In [35]:
query = r'''
    SELECT
        DISTINCT language_code
    FROM books
'''
select(query)

,language_code
0,eng
1,fre
2,spa
3,mul
4,grc
5,ger
6,jpn
7,ara


In [37]:
query = r'''
    SELECT
        language_code, 
        COUNT(*) as books_by_lang
    FROM books
    GROUP BY language_code
'''
select(query)

,language_code,books_by_lang
0,ara,1
1,eng,1420
2,fre,13
3,ger,7
4,grc,7
5,jpn,4
6,mul,9
7,spa,39


In [46]:
query = r'''
    SELECT
        CASE WHEN INSTR(authors, '/') = 0 THEN authors ELSE SUBSTR(authors, 1, INSTR(authors, '/') - 1) END AS author
    FROM books
    LIMIT 10
'''
select(query)

,author
0,J.K. Rowling
1,J.K. Rowling
2,J.K. Rowling
3,J.K. Rowling
4,J.K. Rowling
5,W. Frederick Zimmerman
6,J.K. Rowling
7,Douglas Adams
8,Douglas Adams
9,Douglas Adams


In [48]:
query = r'''
    ALTER TABLE books ADD COLUMN author TEXT;
'''
cursor.executescript(query)
connection.commit()

In [49]:
query = r'''
    UPDATE books
    SET author = CASE WHEN INSTR(authors, '/') = 0 THEN authors ELSE SUBSTR(authors, 1, INSTR(authors, '/') - 1) END 
'''
select(query)

In [53]:
query = r'''
    SELECT 
        author, 
        COUNT(*) count_books
    FROM books
    GROUP BY author
    ORDER BY count_books DESC
'''
select(query)

,author,count_books
0,Euripides,23
1,Aristophanes,21
2,Sophocles,15
3,Mark Twain,15
4,Dan Brown,14
...,...,...
683,Aidan Hartley,1
684,Adam Ginsberg,1
685,Abraham Lincoln,1
686,Abigail Adams,1


In [5]:
pd.read_csv('words.csv').to_sql('words', connection, index=False, if_exists='replace')


5006

In [4]:
connection = sqlite3.connect('words.db')
cursor = connection.cursor()

In [7]:
query = r'''
    SELECT 
        word
    FROM words
    WHERE word GLOB '[Ё-ё]ю*'
    LIMIT 10
    
'''
select(query)

,word
0,бюро
1,июль
2,июнь
3,любезный
4,любимый
5,любитель
6,любить
7,любоваться
8,любовник
9,любовница


### Слово из трех букв

> Напишите запрос, который посчитает в таблице words количество <br>
> слов из трех букв, которые заканчиваются на букву т. Сколько их?

In [11]:
query = r'''
    SELECT 
        COUNT(*) as count_word
    FROM words
    WHERE word LIKE '__т'
'''
select(query)

,count_word
0,12


### Чудные слова

>Напишите запрос, который выберет из таблицы words слова на букву з, у которых вторая буква НЕ а. <br>
>Отсортируйте по популярности по убыванию (столбец freq). Какое слово на втором месте?

In [17]:
query = r'''
    SELECT 
        word
    FROM words
    WHERE word GLOB 'з[^а]*?'
    ORDER BY freq DESC
    LIMIT 3
'''
select(query)

,word
0,знать
1,здесь
2,земля


In [18]:
connection = sqlite3.connect('books.db')
cusror = connection.cursor()

In [19]:
query = r'''
    SELECT * FROM books LIMIT 5
'''
select(query)

,book_id,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_author,author
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,None,J.K. Rowling
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,None,J.K. Rowling
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,None,J.K. Rowling
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,None,J.K. Rowling
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potter...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,None,J.K. Rowling


### Последний книжный день

> Напишите запрос, который по таблице books определит последний день 2005 года, <br>
> когда была опубликована хотя бы одна книга. Что это за день? Укажите ответ в формате гггг-мм-дд.

In [22]:
query = r'''
    SELECT 
        publication_date
    FROM books
    WHERE date(publication_date, 'start of year') = '2005-01-01'
    ORDER BY publication_date DESC
    LIMIT 1
'''
select(query)

,publication_date
0,2005-12-27


## Основные модификаторы:

-    N days — вперед или назад на N дней;
-    N months — вперед или назад на N месяцев;
-    N years — вперед или назад на N лет;
-    start of month — первое число месяца;
-    start of year — первое число года;
-    weekday N — день недели по порядковому номеру (понедельник = 1, вторник = 2, ..., суббота = 6, воскресенье = 0).

### Четвертый четверг

>Напишите запрос, который по таблице books найдет книгу, опубликованную <br>
>в четвертый четверг сентября 2005 года. Укажите в ответе book_id этой книги.

In [49]:
query = r'''
    WITH RECURSIVE dates(date) AS (
    VALUES('2005-09-01')
    UNION ALL
    SELECT date(date, '+1 day')
    FROM dates
    WHERE date < '2005-09-30'
    ), get_fourth_day AS (
    SELECT 
        date,
        ROW_NUMBER() OVER w as number
    FROM dates
    WHERE date = date(date, 'weekday 4')
    WINDOW w AS (
        ORDER BY date
    )
    ), get_fourth_thursday AS (
    SELECT 
        date AS fourth_thursday
    FROM get_fourth_day
    WHERE number = 4
    )
    SELECT 
        book_id
    FROM books, get_fourth_thursday
    WHERE publication_date = fourth_thursday
'''
select(query)

,book_id
0,1241


### Слова популярные и не очень

Посчитайте среднюю популярность (столбец freq) по всем словам в таблице words. Округлите до целого значения. Получилось некоторое значение N. Теперь отнесите каждое слово к одной из двух категорий:

    rare — если популярность слова меньше N,
    frequent — если популярность слова N или больше.

Используйте функцию iif(), чтобы определить категорию для каждого слова.

Посчитайте количество frequent-слов, которые начинаются на букву я. Сколько их?

In [52]:
connection = sqlite3.connect('words.db')
cusror = connection.cursor()

In [61]:
query = r'''
    WITH get_cat AS (
    SELECT 
        word,
        CASE WHEN AVG(freq) OVER w > freq THEN 'rare' ELSE 'frequent' END AS category
    FROM words
    WINDOW w AS (
        ROWS BETWEEN unbounded preceding AND unbounded following
    ))
    SELECT 
        COUNT(*)
    FROM get_cat
    WHERE category = 'frequent' AND word LIKE 'я%'
'''
select(query)

,COUNT(*)
0,2


In [63]:
query = r'''
    SELECT coalesce(nullif('a', 'z'), 'oh my')
'''
select(query)

,"coalesce(nullif('a', 'z'), 'oh my')"
0,a


## Как почистить набор данных:

-    Создать таблицу с правильными типами столбцов.
-    Загрузить данные.
-    Привести проблемные значения к правильным типам.
-    Занулить мусорные значения.
-    Заполнить пробелы в данных.
-    Выгрузить чистые данные.
